# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [22]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [23]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mount isa,-20.7333,139.5000,25.87,83,40,4.12,AU,2024-02-13
1,1,port-aux-francais,-49.3500,70.2167,8.16,91,100,10.98,TF,2024-02-13
2,2,puerto ayora,-0.7393,-90.3518,26.42,100,78,2.24,EC,2024-02-13
3,3,albany,42.6001,-73.9662,0.49,78,100,0.89,US,2024-02-13
4,4,hermanus,-34.4187,19.2345,23.78,71,39,5.14,ZA,2024-02-13


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [24]:
# Configure the map
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 600,
    size = "Humidity",
    alpha = 0.5,
    color = "City"
)

# Display the map plot
map_plot_1

c:\Users\james\anaconda3\envs\firstpythonenv\lib\site-packages\cartopy\crs.py:245: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  if len(multi_line_string) > 1:
c:\Users\james\anaconda3\envs\firstpythonenv\lib\site-packages\cartopy\crs.py:297: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for line in multi_line_string:
c:\Users\james\anaconda3\envs\firstpythonenv\lib\site-packages\cartopy\crs.py:364: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  if len(p_mline) > 0:
c:\Users\james\anaconda3\envs\firstpy

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [25]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_locations_df = city_data_df[(city_data_df['Lat'] >= -30) & (city_data_df['Lat'] <= -15)]


# Check for null values in the DataFrame
null_values = ideal_weather_locations_df.isnull()

# Display the DataFrame with True where values are null and False where values are not null
null_count_per_column = ideal_weather_locations_df.isnull().sum()

print(null_count_per_column)


City_ID       0
City          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
Country       5
Date          0
dtype: int64


In [26]:
# Drop any rows with null values
ideal_weather_locations_df_without_nulls = ideal_weather_locations_df.dropna()

# Display sample data
ideal_weather_locations_df_without_nulls.sample(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
260,260,presidente epitacio,-21.7633,-52.1156,34.56,38,41,4.07,BR,2024-02-13
437,437,puerto quijarro,-17.7833,-57.7667,27.81,72,100,2.37,BO,2024-02-13
177,177,sao joao da barra,-21.6403,-41.0511,32.05,50,95,2.53,BR,2024-02-13
460,460,santa isabel,-23.3156,-46.2214,29.61,49,100,4.87,BR,2024-02-13
284,284,emerald,-23.5333,148.1667,24.08,87,100,5.20,AU,2024-02-13
89,89,avarua,-21.2078,-159.7750,23.03,94,76,0.51,CK,2024-02-13
295,295,santa terezinha,-29.9002,-50.0904,24.19,99,100,1.38,BR,2024-02-13
433,433,antofagasta,-23.6500,-70.4000,22.60,72,68,2.75,CL,2024-02-13
225,225,suva,-18.1416,178.4415,26.71,94,20,1.54,FJ,2024-02-13
267,267,tsiombe,-25.3000,45.4833,27.22,74,37,8.53,MG,2024-02-13


### Step 3: Create a new DataFrame called `hotel_df`.

In [27]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_locations_df_without_nulls.copy()[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.sample(10)

,City,Country,Lat,Lng,Humidity,Hotel Name
260,presidente epitacio,BR,-21.7633,-52.1156,38,
528,alice springs,AU,-23.7000,133.8833,25,
390,kone,NC,-21.0595,164.8658,90,
185,nuku'alofa,TO,-21.1333,-175.2000,83,
399,orkney,ZA,-26.9802,26.6727,14,
445,terra roxa,BR,-23.8557,-50.0314,40,
380,tolanaro,MG,-25.0319,46.9987,100,
102,port mathurin,MU,-19.6833,63.4167,82,
437,puerto quijarro,BO,-17.7833,-57.7667,72,
151,geraldton,AU,-28.7667,114.6000,73,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [28]:
# Set parameters to search for a hotel
radius = 10000
limit = 1
params = {
    'categories': 'accommodation.hotel',
    'limit': limit,
    'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")


Starting hotel search
mount isa - nearest hotel: Ibis Styles
adamstown - nearest hotel: No hotel found
minas de marcona - nearest hotel: No hotel found
uturoa - nearest hotel: Hawaiki Nui hotel
morondava - nearest hotel: Hotel Sharon Menabe
luganville - nearest hotel: Beachfront Resort
nadi - nearest hotel: President Hotel
avarua - nearest hotel: Paradise Inn
ghanzi - nearest hotel: Ghanzi farmhouse
port mathurin - nearest hotel: Escale Vacances
tadine - nearest hotel: Hôtel Nengoné
arraial do cabo - nearest hotel: No hotel found
padre paraiso - nearest hotel: No hotel found
ritchie - nearest hotel: No hotel found
afaahiti - nearest hotel: Omati Lodge
geraldton - nearest hotel: Ocean Centre Hotel
nova vicosa - nearest hotel: Pousada Pontal da Barra
alofi - nearest hotel: Matavai Resort
sao joao da barra - nearest hotel: Pousada Mediterrâneo
nuku'alofa - nearest hotel: City Hotel
carnarvon - nearest hotel: No hotel found
chokwe - nearest hotel: Limpopo Hotel
colonia menno - nearest hote

In [29]:
# Display sample data
hotel_df.sample(10)

,City,Country,Lat,Lng,Humidity,Hotel Name
455,kariba,ZW,-16.5167,28.8000,42,Lodge
412,coquimbo,CL,-29.9533,-71.3436,100,Hotel Iberia
399,orkney,ZA,-26.9802,26.6727,14,No hotel found
225,suva,FJ,-18.1416,178.4415,94,Extension 68
185,nuku'alofa,TO,-21.1333,-175.2000,83,City Hotel
442,karratha,AU,-20.7377,116.8463,78,Karratha International Hotel
386,urangan,AU,-25.3000,152.9000,93,Ramada
437,puerto quijarro,BO,-17.7833,-57.7667,72,No hotel found
476,santa cruz,BO,-17.8000,-63.1667,94,Aviador Hotel Boutique
380,tolanaro,MG,-25.0319,46.9987,100,Hôtel Mahavokey


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [31]:
# Configure the map
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 600,
    size = "Humidity",
    alpha = 0.5,
    color = "City",
    hover_cols=["Country", "Hotel Name"]
)

# Display the map plot
map_plot_2

c:\Users\james\anaconda3\envs\firstpythonenv\lib\site-packages\cartopy\crs.py:245: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  if len(multi_line_string) > 1:
c:\Users\james\anaconda3\envs\firstpythonenv\lib\site-packages\cartopy\crs.py:297: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for line in multi_line_string:
c:\Users\james\anaconda3\envs\firstpythonenv\lib\site-packages\cartopy\crs.py:364: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  if len(p_mline) > 0:
c:\Users\james\anaconda3\envs\firstpy

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)